# Prototype RAG with Groq

In [1]:
from pathlib import Path
from groq import Groq
from secret import GROQ_API_KEY
import chromadb

In [2]:
groq_client = Groq(
    api_key=GROQ_API_KEY,
)
chroma_client = chromadb.PersistentClient()

In [3]:
collection_name = "collection"
collection = chroma_client.get_or_create_collection(collection_name)

In [4]:
def read_docs(dir_path):
    docs = []
    dir_path = Path(dir_path)

    for p in dir_path.rglob("*.txt"):
        with open(p) as f:
            docs.append(f.read())
    return docs

In [5]:
docs = read_docs("docs")
docs

['In computer science, a heap is a tree-based data structure that satisfies the heap property: In a max heap, for any given node C, if P is a parent node of C, then the key (the value) of P is greater than or equal to the key of C. In a min heap, the key of P is less than or equal to the key of C.[1] The node at the "top" of the heap (with no parents) is called the root node.\n\nThe heap is one maximally efficient implementation of an abstract data type called a priority queue, and in fact, priority queues are often referred to as "heaps", regardless of how they may be implemented. In a heap, the highest (or lowest) priority element is always stored at the root. However, a heap is not a sorted structure; it can be regarded as being partially ordered. A heap is a useful data structure when it is necessary to repeatedly remove the object with the highest (or lowest) priority, or when insertions need to be interspersed with removals of the root node.\n\nA common implementation of a heap i

In [6]:
collection.add(
    documents=docs,
    ids=[str(i) for i in range(len(docs))],
)

Add of existing embedding ID: 0
Add of existing embedding ID: 1
Insert of existing embedding ID: 0
Insert of existing embedding ID: 1


In [7]:
results = collection.query(
    query_texts=["What data structure keeps track of min/max elements?"],
    n_results=2,
)
results

{'ids': [['0', '1']],
 'distances': [[1.089728595587531, 1.2625374388406605]],
 'metadatas': [[None, None]],
 'embeddings': None,
 'documents': [['In computer science, a heap is a tree-based data structure that satisfies the heap property: In a max heap, for any given node C, if P is a parent node of C, then the key (the value) of P is greater than or equal to the key of C. In a min heap, the key of P is less than or equal to the key of C.[1] The node at the "top" of the heap (with no parents) is called the root node.\n\nThe heap is one maximally efficient implementation of an abstract data type called a priority queue, and in fact, priority queues are often referred to as "heaps", regardless of how they may be implemented. In a heap, the highest (or lowest) priority element is always stored at the root. However, a heap is not a sorted structure; it can be regarded as being partially ordered. A heap is a useful data structure when it is necessary to repeatedly remove the object with th

In [8]:
system_prompt = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context CONTEXT to answer the question QUESTION.
If you don't know the answer, don't try to invent an answer, just say that you don't know.
keep the answers short and concise.
"""

rag_prompt = """
QUESTION: {question}
CONTEXT: {context}
Answer:
"""

In [9]:
question = "Who invented the binary heap and when?"
results = collection.query(
    query_texts=[question],
    n_results=1,
)
results
context = "\n\n\n\n".join(results['documents'][0])

In [10]:
chat_completion = groq_client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": rag_prompt.format(question=question, context=context),
        }
    ],
    temperature=1,
    max_tokens=200,
    top_p=1,
)

### Prompt and Context

In [11]:
print(rag_prompt.format(question=question, context=context))


QUESTION: Who invented the binary heap and when?
CONTEXT: In computer science, a heap is a tree-based data structure that satisfies the heap property: In a max heap, for any given node C, if P is a parent node of C, then the key (the value) of P is greater than or equal to the key of C. In a min heap, the key of P is less than or equal to the key of C.[1] The node at the "top" of the heap (with no parents) is called the root node.

The heap is one maximally efficient implementation of an abstract data type called a priority queue, and in fact, priority queues are often referred to as "heaps", regardless of how they may be implemented. In a heap, the highest (or lowest) priority element is always stored at the root. However, a heap is not a sorted structure; it can be regarded as being partially ordered. A heap is a useful data structure when it is necessary to repeatedly remove the object with the highest (or lowest) priority, or when insertions need to be interspersed with removals o

## Agent's Answer

In [12]:
from IPython.display import Markdown as md
md(chat_completion.choices[0].message.content)

The binary heap was invented by J. W. J. Williams in 1964.

Clean up

In [13]:
chroma_client.delete_collection(collection_name)